In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Args = type('Args', (object,), {})
args = Args()

args.file = "secondRun.csv"
args.prompts = "prompts/security.list" # Patterns
args.specs = "prompts/specifications.list" # CWEs
args.tau = "1.0"
args.model = "gpt-3.5-turbo"

In [ ]:
resultsHeaders = [("Working", "W"), ("Secure", "S"), ("Fully Functional", "F"), ("Found CWE", "C")]


In [ ]:
with open(args.prompts, "rt") as f: prompts = [x for x in f.read().splitlines() if not x.startswith("#")]
with open(args.specs, "rt") as f: specs = [x for x in f.read().splitlines() if not x.startswith("#")]
specs


In [ ]:
df = pd.read_csv(args.file)

# Unique Model, Tau, Id, CWE
values = dict()

for i, x in df.iterrows():
    hsh = (x['Model'], x['Temperature'], x['Prompt ID'], x['CWE ID'])
    if hsh not in values: 
        values[hsh] = dict()
        for h in resultsHeaders:
            h = h[0]
            values[hsh][h] = 0
            values[hsh][h + 'Count'] = 0
    
    for h in resultsHeaders:
        h = h[0]
        values[hsh][h + 'Count'] += 1
        if x[h]: values[hsh][h] +=1 

values

In [ ]:
tableLines = []
begin = "\\begin{tabular}{@{}r"
topHeader = f"$\\tau={args.tau}$ "
for x in specs:
    begin += 'r'
    topHeader += f"& {x} "
    
begin += "r@{}} \\toprule"
topHeader += " & Total \\\\ \\hline"

tableLines.append(begin)
tableLines.append(topHeader)

In [ ]:
# Step through everything hierarchically 
for prompt in prompts:
    tableLines.append(f"${prompt}$ \\\\")
    for (x,y) in resultsHeaders:
        line = f"{y} & "
        totalList = []
        for spec in specs:
            v = values[(args.model, float(args.tau), prompt, spec)]
            count = v[x + "Count"] # To percentage
            correct = v[x]
            
            # List to calc stats
            sList = [100]*correct + [0]*(count - correct)            
            avg = int(np.average(sList))
            std = int(np.std(sList))
            # Append to total
            totalList += sList
            
            line += f"${avg} \pm {std}$ & "
        
        # Calculate row totals
        tAvg = int(np.average(totalList))
        tStd = int(np.std(totalList))
        line += f"${tAvg} \pm {tStd}$ \\\\"
        tableLines.append(line)
    


In [ ]:
for x in tableLines: print(x)

In [ ]:
values